In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curre

In [29]:
model_name = "NousResearch/llama-2-7b-chat-hf" # use this if you have access to the official LLaMA 2 model "meta-llama/Llama-2-7b-chat-hf", though keep in mind you'll need to pass a Hugging Face key argument
dataset_name = "/content/train.jsonl"
new_model = "llama-2-7b-custom"
lora_r = 32
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 5
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"": 0}

In [30]:
dataset_name = "machine_learning"

In [31]:
def get_choices():
    return ["A", "B", "C", "D"]


def format_subject(subject):
    return subject.replace("_", " ")


def format_example(example, include_answer=True):
    # Extract question and choices
    prompt = example['question']
    choices = example['choices']

    # Add choices to the prompt
    for j, choice in enumerate(choices):
        prompt += "\n{}. {}".format(get_choices()[j], choice)

    prompt += "\nAnswer:"

    if include_answer:
        # Get the answer (convert to letter if it's an index)
        answer = example['answer']
        if isinstance(answer, (int, np.integer)):
            answer = get_choices()[answer]
        prompt += " {}\n\n".format(answer)

    return prompt

In [32]:
system_message = f"You are an expert in {format_subject(dataset_name)}. You will be presented with a question and four possible answers. Choose the correct answer. Do not output any explanation, only choose the correct option."

In [33]:
# Load datasets
train_dataset = load_dataset('json', data_files=f'/content/{dataset_name}_train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files=f'/content/{dataset_name}_test.jsonl', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)

In [34]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [35]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [36]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5  # Evaluate every 20 steps
)

In [37]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/23 [00:00<?, ? examples/s]

In [38]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
5,3.202500,2.628539
10,2.311800,1.873727
15,1.645900,1.369977
20,1.238000,1.073732
25,0.923900,0.935749
30,0.810000,0.893921
35,0.709100,0.858677
40,0.790700,0.789148
45,0.740300,0.738643
50,0.628600,0.724271


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

KeyboardInterrupt: 

In [13]:
trainer.model.save_pretrained(new_model)

In [39]:
dataset_name = "machine_learning"

In [40]:
from datasets import load_dataset
ds = load_dataset("cais/mmlu", dataset_name)

In [41]:
system_prompt = f"You are an expert in {format_subject(dataset_name)}. You will be presented with a question and four possible answers. Choose the correct answer. Do not output any explanation, only choose the correct option."

In [42]:
import re

def extract_answer(generated_text):
    """
    Extract the first letter that matches A, B, C, or D
    """
    # Convert to uppercase to catch both lower and upper case
    generated_text = generated_text.upper()

    # Look for first occurrence of A, B, C, or D
    match = re.search(r'[A-D]', generated_text)

    return match.group(0) if match else None

In [43]:
num_few_shot = 0
total_questions = 100

In [44]:
import numpy as np
from tqdm import tqdm

In [45]:
correct_predictions = 0

correct_answers = []
predicted_answers = []

import time

t1 = time.time()

for idx in tqdm(range(total_questions)):
    # Current test question
    test_question = format_example(ds['test'][idx], include_answer=False)

    prompt = f"""[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n {test_question} [/INST]"""

    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1024)

    result = pipe(prompt)

    text = result[0]['generated_text']

    content = text.split("[/INST]")[1].strip()

    predicted_answer = extract_answer(content)

    ground_truth = ds['test'][idx]['answer']

    # Normalize ground truth
    if isinstance(ground_truth, (int, np.integer)):
        ground_truth = get_choices()[ground_truth]

    predicted_answers.append(predicted_answer)
    correct_answers.append(ground_truth)

    # Compare
    if predicted_answer == ground_truth:
        correct_predictions += 1
    # else:
    #     # Optional: print misclassified examples for debugging
    #     print(f"Predicted: {predicted_answer}, Correct: {ground_truth}")

t2 = time.time()

accuracy = correct_predictions / total_questions
print(f"Accuracy: {accuracy}")

print(f"Time taken: {t2-t1}")

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 100/100 [06:29<00:00,  3.89s/it]

Accuracy: 0.22
Time taken: 389.3747606277466


In [46]:
import pandas as pd
df = pd.DataFrame({
    "Question": [format_example(ds['test'][i], include_answer=False) for i in range(total_questions)],
    "Predicted Answer": predicted_answers,
    "Correct Answer": correct_answers
})

df.to_csv(f"{dataset_name}_llama_finetuned-(machine_learning)_results.csv", index=False)

In [47]:
dataset_name = "professional_psychology"

In [48]:
ds = load_dataset("cais/mmlu", dataset_name)

In [49]:
system_prompt = f"You are an expert in {format_subject(dataset_name)}. You will be presented with a question and four possible answers. Choose the correct answer. Do not output any explanation, only choose the correct option."

In [50]:
correct_predictions = 0

correct_answers = []
predicted_answers = []

import time

t1 = time.time()

for idx in tqdm(range(total_questions)):
    # Current test question
    test_question = format_example(ds['test'][idx], include_answer=False)

    prompt = f"""[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n {test_question} [/INST]"""

    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1024)

    result = pipe(prompt)

    text = result[0]['generated_text']

    content = text.split("[/INST]")[1].strip()

    predicted_answer = extract_answer(content)

    ground_truth = ds['test'][idx]['answer']

    # Normalize ground truth
    if isinstance(ground_truth, (int, np.integer)):
        ground_truth = get_choices()[ground_truth]

    predicted_answers.append(predicted_answer)
    correct_answers.append(ground_truth)

    # Compare
    if predicted_answer == ground_truth:
        correct_predictions += 1
    # else:
    #     # Optional: print misclassified examples for debugging
    #     print(f"Predicted: {predicted_answer}, Correct: {ground_truth}")

t2 = time.time()

accuracy = correct_predictions / total_questions
print(f"Accuracy: {accuracy}")

print(f"Time taken: {t2-t1}")

100%|██████████| 100/100 [09:42<00:00,  5.82s/it]

Accuracy: 0.31
Time taken: 582.4302875995636


In [51]:
df = pd.DataFrame({
    "Question": [format_example(ds['test'][i], include_answer=False) for i in range(total_questions)],
    "Predicted Answer": predicted_answers,
    "Correct Answer": correct_answers
})

df.to_csv(f"{dataset_name}_llama_finetuned-(machine_learning)_results.csv", index=False)